In [ ]:
# install pydrake

# try:
#   import pydrake
#   import underactuated
# except ImportError:
#   !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
#   from jupyter_setup import setup_underactuated
#   setup_underactuated()


In [ ]:
from pydrake.all import eq, MathematicalProgram, Solve, Variable

import matplotlib.pyplot as plt
import numpy as np
import importlib

from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator

from pydrake.all import (
    DiagramBuilder, Simulator
)

from pydrake.multibody.tree import (
    JointActuatorIndex
)

from pydrake.geometry import SceneGraph
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.trajectories import PiecewisePolynomial
from pydrake.solvers.snopt import SnoptSolver

import kinematic_constraints
import dynamics_constraints

In [ ]:
# set up constants

N = 7      # number of collocation points
n_x = 10
n_u = 3

# where we start, where we go, mass?, 
# start position
alpha_0 = -23.45   #longitude degrees
beta_0 = -2.94     #latitude degrees
r_0 = 3474.8       #radius of Mars, in km
phi_0 = 90         #pitch degrees
psi_0 = 0          #yaw degrees

# final position
alpha_N = -23.45   #degrees
beta_N = -1.94     #degrees
r_N = 3474.8       #radius of Mars, in km
phi_N = 90
psi_N = 0

# initial control input 
T_0 = 0
phiddot_N = 0
psiddot_N = 0

# final control input
T_N = 0
phiddot_N = 0
psiddot_N = 0

# constraint coefficients
r_min = 3474.8
r_max = 3474.8 + 30 #30km above Mars surface
phi_min = -20      #degree
phi_max = 90       #degree
psi_min = -180     #degree
psi_max = 180      #degree
phidot_min = -8    #deg/sec
phidot_max = 8     #deg/sec
psidot_min = -8    #deg/sec
psidot_max = 8     #deg/sec
T_min = 4671       #Newton
T_max = 43148      #Newton

m = 8321.09        #kg
g = 3.711          #m/s^2


In [ ]:
prog = MathematicalProgram()
x = np.zeros((N,n_x),dtype="object")
u = np.zeros((N,n_u),dtype="object")
# t = np.zeros((N,1),dtype="object")


for i in range(N):
    x[i] = prog.NewContinuousVariables(n_x,"x_" + str(i))
    u[i] = prog.NewContinuousVariabels(n_u,"u_" + str(i))
#     t[i] = prog.NewContinuousVariables(1,"t_" + str(i))
t_land = prog.NewContinuousVariables(1,"t_land")

t0 = 0.0
timesteps = np.linspace(t0,t_land,N)


In [ ]:
# TODO: Add cost (sum of square of all inputs)

In [ ]:
# TODO: Add dynamic constraint

In [ ]:
# TODO: Add kinematic constraint

In [ ]:
# TODO: Add input/state limitation as bounding box constraints
# 0 <= h <= hmax
prog.AddBoundingBoxConstraint(r_min,r_max,x[:,2])

# omega angular velocity state limits
prog.AddBoundingBoxConstraint(phidot_min,phidot_max,x[:,8])
prog.AddBoundingBoxConstraint(psidot_min,psidot_max,x[:,9])

# angular position state limit
prog.AddBoundingBoxConstraint(phi_min,phi_max,x[:,3])
prog.AddBoundingBoxConstraint(psi_min,psi_max,x[:,4])

# thrust limit
prog.AddBoundingBoxConstraint(T_min,T_max,u[:,0])

# maybe we need to add phiddot and psiddot limit, but not a priority

In [ ]:
# TODO: Set up solver initial guess

uval = np.zeros((N,n_u))
uval[:,0] = np.random.uniform(T_min,T_max,N)
#     uval[:,1] = np.random.uniform()

In [ ]:
# Set up solver

solver = SnoptSolver()
result = solver.Solve(prog)
  
x_sol = result.GetSolution(x)
u_sol = result.GetSolution(u)
t_sol = result.GetSolution(t)



In [ ]:
# TODO: create LQR to follow the trajectory (use HW2)

In [ ]:
# TODO: simulate and animate path, create disturbance in simulation